In [3]:
# ================================================================
# 모듈 9: 부문 단위 CL별 정규화 모듈 (Supervisor 기반)
# ================================================================

# 1. 환경 설정 및 라이브러리 설치 (필요시)
# !pip install langchain langchain-openai langgraph sqlalchemy pymysql python-dotenv

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../..')))

from config.settings import DatabaseConfig
from dotenv import load_dotenv
load_dotenv()

# 2. 필수 라이브러리 임포트
from typing import Annotated, List, Literal, TypedDict, Dict, Any, Optional
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import operator
from langgraph.graph import StateGraph, START, END
import json
import re
import statistics
import time

# SQLAlchemy 관련 임포트
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Connection, Row

# LangChain LLM 관련 임포트
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 3. 설정 로드
print("🚀 모듈9 초기화 중...")

# DB 설정
db_config = DatabaseConfig()
DATABASE_URL = db_config.DATABASE_URL
engine = create_engine(DATABASE_URL, pool_pre_ping=True)

# LLM 클라이언트 설정
llm_client = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(f"LLM Client initialized: {llm_client.model_name}")

# 4. 도우미 함수들
def row_to_dict(row: Row) -> Dict[str, Any]:
    """SQLAlchemy Row 객체를 딕셔너리로 변환"""
    if row is None:
        return {}
    return row._asdict()

def extract_json_from_llm_response(text: str) -> str:
    """LLM 응답에서 JSON 블록 추출"""
    match = re.search(r"```(?:json)?\s*(.*?)\s*```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

# ================================================================
# Module9AgentState 정의
# ================================================================

class Module9AgentState(TypedDict):
    """모듈 9 (부문 단위 CL별 정규화) 상태"""
    messages: Annotated[List[HumanMessage], operator.add]
    
    # 입력 정보
    headquarter_id: int
    period_id: int  # 연말: 4
    
    # 서브모듈별 결과 (간소하게)
    department_data: Dict[str, Dict]  # 1단계: 부문 데이터 수집 결과
    supervisor_results: Dict[str, Dict]  # 2단계: Supervisor 실행 결과  
    update_results: Dict  # 3단계: 배치 업데이트 결과
    
    # 처리 상태
    total_processed: int
    total_failed: int
    error_logs: List[str]

# ================================================================
# 프롬프트 관리 함수들
# ================================================================

def get_prompt_by_id(prompt_id: int) -> str:
    """프롬프트 ID로 프롬프트 내용 조회"""
    with engine.connect() as connection:
        query = text("SELECT prompt FROM prompts WHERE prompt_id = :prompt_id")
        result = connection.execute(query, {"prompt_id": prompt_id}).scalar()
        return result if result else ""

def get_config_by_headquarter(headquarter_id: int) -> Optional[str]:
    """본부별 설정값 조회 (없으면 None 반환)"""
    # 향후 본부별 커스터마이징 구현 시 사용
    return None

def build_supervisor_prompt(headquarter_id: int, cl_group: str, required_adjustment: float) -> str:
    """설정값을 반영한 동적 프롬프트 생성"""
    
    # 기본 설정값 (프롬프트 테이블에서 조회 가능)
    default_config = {
        "performance_weight": 50,
        "team_weight": 30, 
        "special_weight": 20,
        "min_score": 1.5,
        "max_score": 5.0,
        "gap_threshold": 10
    }
    
    # 본부별 설정 조회 (없으면 기본값 사용)
    config_text = get_config_by_headquarter(headquarter_id)
    if config_text:
        try:
            config = json.loads(config_text)
            default_config.update(config)
        except:
            pass  # 파싱 실패 시 기본값 사용
    
    # 시스템 프롬프트 구성
    system_prompt = f"""당신은 조직의 공정하고 합리적인 성과 조정 전문가입니다.
팀장들의 점수 조정 요청을 검토하고, 전체적 균형을 고려한 최적 조정안을 도출하세요.

평가 우선순위 (중요도 순):
1. KPI 달성률 (최우선) - 연간 달성률, 참여 KPI 성과
2. 개인 기여도 - 연간 기여도, 업무 품질
3. 팀/조직 맥락 - 팀 성과, 업무 난이도, 협업 기여  
4. 특수 상황 - 리더십 역할, 멘토링, 위기 대응

평가 가중치:
- KPI 달성률 및 성과 지표 ({default_config['performance_weight']}%): 연간 달성률, 참여 KPI 성과, 업무 품질
- 팀/조직 맥락 ({default_config['team_weight']}%): 팀 성과, 업무 난이도, 협업 기여  
- 특수 상황 ({default_config['special_weight']}%): 리더십 역할, 멘토링, 위기 대응

제약 조건:
- [필수] 최종 점수 {default_config['min_score']}~{default_config['max_score']}점 범위 준수
- [필수] 성과 역전 방지 (KPI 달성률 차이 {default_config['gap_threshold']}%p 이상 시 점수 역전 불가)
- [필수] 총 차감량 {required_adjustment:.2f}점 정확히 달성 (제로섬)
- [권장] 개인별 변화폭 ±1.0점 이내 권장

예외 처리:
- 조정 불가 시: 상승 요청 50%까지 축소 허용
- 제로섬 미달성 시: 남은 차감량 명시하여 리턴
- 제약 위반 시: 제약 조건 우선, 상세 사유 기록

맥락 고려사항:
- 점수 출처별 차등 평가 필요:
  * raw_score_preserved: 1명 팀으로 원점수 보존됨 (정규화 안됨) → 하향 조정 우선 고려
  * minimal_normalized: 2-3명 소규모 팀으로 최소 정규화만 적용됨
  * normalized: 4명 이상 팀으로 정상 정규화 적용됨 
  * captain_adjusted: 팀장이 의도적으로 수정함 → 팀장 의도 존중
- 팀 크기 맥락 (single_member_team/small_team/normal_team)을 조정 근거에 포함
- raw_score_preserved 점수는 이미 과도하게 높을 가능성 고려
- 모든 조정은 개인 맥락과 조직 전체 관점 균형 고려

결과는 다음 JSON 형식으로만 응답하세요. 불필요한 서문이나 추가 설명 없이 JSON만 반환해야 합니다."""

    return system_prompt

# ================================================================
# 데이터 조회 함수들
# ================================================================

def fetch_headquarter_cl_data(headquarter_id: int, period_id: int) -> Dict[str, List[Dict]]:
    """본부 내 모든 직원 데이터를 CL별로 조회"""
    
    with engine.connect() as connection:
        query = text("""
            SELECT 
                e.emp_no, e.emp_name, e.cl, e.position, e.team_id,
                te.score as current_score,
                te.reason as captain_reason,
                (SELECT COUNT(*) FROM employees e2 WHERE e2.team_id = e.team_id AND e2.cl = e.cl) as cl_team_size,
                CASE 
                    WHEN te.reason IS NOT NULL AND te.reason != '' THEN 'captain_adjusted'
                    WHEN (SELECT COUNT(*) FROM employees e2 WHERE e2.team_id = e.team_id AND e2.cl = e.cl) <= 1 THEN 'raw_score_preserved'
                    WHEN (SELECT COUNT(*) FROM employees e2 WHERE e2.team_id = e.team_id AND e2.cl = e.cl) <= 3 THEN 'minimal_normalized'
                    ELSE 'normalized'
                END as score_origin,
                CASE 
                    WHEN (SELECT COUNT(*) FROM employees e2 WHERE e2.team_id = e.team_id AND e2.cl = e.cl) = 1 THEN 'single_member_team'
                    WHEN (SELECT COUNT(*) FROM employees e2 WHERE e2.team_id = e.team_id AND e2.cl = e.cl) <= 3 THEN 'small_team'
                    ELSE 'normal_team'
                END as team_size_context
            FROM employees e
            JOIN teams t ON e.team_id = t.team_id
            JOIN temp_evaluations te ON e.emp_no = te.TempEvaluation_empNo
            WHERE t.headquarter_id = :headquarter_id
            AND te.status = '완료'
            ORDER BY e.cl DESC, e.emp_no
        """)
        
        results = connection.execute(query, {
            "headquarter_id": headquarter_id
        }).fetchall()
        
        # CL별 그룹화
        cl_groups = {"CL3": [], "CL2": [], "CL1": []}
        
        for row in results:
            data = row_to_dict(row)
            cl_key = f"CL{data['cl']}"
            if cl_key in cl_groups:
                cl_groups[cl_key].append(data)
        
        return cl_groups

def fetch_employee_detailed_data(emp_no: str, period_id: int) -> Dict:
    """개별 직원의 상세 성과 데이터 조회"""
    
    with engine.connect() as connection:
        # 기본 성과 지표 조회
        query = text("""
            SELECT 
                fer.contribution_rate,
                fer.ai_annual_achievement_rate as kpi_achievement,
                fer.ai_peer_talk_summary,
                fer.ai_4p_evaluation
            FROM final_evaluation_reports fer
            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
            WHERE fer.emp_no = :emp_no AND te.period_id = :period_id
        """)
        
        result = connection.execute(query, {
            "emp_no": emp_no,
            "period_id": period_id
        }).fetchone()
        
        if result:
            data = row_to_dict(result)
            
            # 4P 평가 JSON 파싱
            if data.get('ai_4p_evaluation'):
                try:
                    data['fourp_evaluation'] = json.loads(data['ai_4p_evaluation'])
                except:
                    data['fourp_evaluation'] = {}
            else:
                data['fourp_evaluation'] = {}
            
            return data
        
        return {}

def fetch_employee_participated_kpis(emp_no: str, period_id: int) -> List[Dict]:
    """개별 직원의 참여 KPI 성과 조회"""
    
    with engine.connect() as connection:
        query = text("""
            SELECT 
                tk.kpi_name,
                tk.ai_kpi_progress_rate,
                tk.ai_kpi_analysis_comment,
                tk.weight
            FROM tasks t
            JOIN team_kpis tk ON t.team_kpi_id = tk.team_kpi_id
            JOIN task_summaries ts ON t.task_id = ts.task_id
            WHERE ts.period_id = :period_id
            AND t.emp_no = :emp_no
            GROUP BY t.emp_no, tk.team_kpi_id
        """)
        
        results = connection.execute(query, {
            "emp_no": emp_no,
            "period_id": period_id
        }).fetchall()
        
        return [row_to_dict(row) for row in results]

def calculate_cl_surplus(cl_members: List[Dict]) -> float:
    """CL 그룹의 초과 점수 계산"""
    if not cl_members:
        return 0.0
    
    member_count = len(cl_members)
    target_total = member_count * 3.5  # 목표 총점
    current_total = sum(member.get('current_score', 3.5) for member in cl_members)
    
    return round(current_total - target_total, 2)

# ================================================================
# Supervisor LLM 호출 함수들
# ================================================================

def call_supervisor_llm(cl_data: Dict, headquarter_id: int, period_id: int) -> Dict:
    """Supervisor LLM 호출"""
    
    cl_group = cl_data['cl_group']
    members = cl_data['members']
    required_adjustment = cl_data['surplus']
    
    print(f"🤖 Supervisor LLM 호출: {cl_group} (조정 필요: {required_adjustment:.2f}점)")
    
    # 시스템 프롬프트 생성
    system_prompt = build_supervisor_prompt(headquarter_id, cl_group, required_adjustment)
    
    # 입력 데이터 구성
    members_data = []
    for member in members:
        emp_no = member['emp_no']
        
        # 상세 데이터 조회
        detailed_data = fetch_employee_detailed_data(emp_no, period_id)
        participated_kpis = fetch_employee_participated_kpis(emp_no, period_id)
        
        member_input = {
            "emp_no": emp_no,
            "emp_name": member.get('emp_name'),
            "position": member.get('position'),
            "current_score": member.get('current_score'),
            "captain_reason": member.get('captain_reason'),
            "score_origin": member.get('score_origin'),
            "team_size_context": member.get('team_size_context'),
            "cl_team_size": member.get('cl_team_size'),
            "kpi_achievement": detailed_data.get('kpi_achievement', 0),
            "contribution_rate": detailed_data.get('contribution_rate', 0),
            "peer_talk_summary": detailed_data.get('ai_peer_talk_summary', ''),
            "fourp_evaluation": detailed_data.get('fourp_evaluation', {}),
            "participated_kpis": participated_kpis
        }
        members_data.append(member_input)
    
    # Human 프롬프트 구성
    human_prompt = f"""
{cl_group} 그룹 조정 요청 분석

현황:
- 목표 평균: 3.5점
- 현재 총점: {sum(m['current_score'] for m in members):.1f}점
- 목표 총점: {len(members) * 3.5:.1f}점  
- 필요 조정: {required_adjustment:.2f}점

조정 대상 인원: {json.dumps(members_data, ensure_ascii=False, indent=2)}

다음 순서로 분석하여 JSON 응답:
1. 각 상승 요청의 타당성 검증
2. 최적 조정안 도출 (상승/차감/유지)
3. 제로섬 달성 확인

JSON 응답:
{{
  "analysis_summary": "전체 상황 분석 요약",
  "adjustments": [
    {{
      "emp_no": "E001",
      "original_score": 3.2,
      "final_score": 3.8,
      "change_amount": 0.6,
      "change_type": "increase|decrease|maintain",
      "reason": "상세 조정 사유"
    }}
  ],
  "zero_sum_check": {{
    "target_adjustment": {required_adjustment:.2f},
    "actual_adjustment": -1.3,
    "achieved": true
  }},
  "rejected_requests": [
    {{
      "emp_no": "E002", 
      "requested_increase": 0.8,
      "rejection_reason": "성과 대비 과도한 요청"
    }}
  ]
}}
"""
    
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt)
    ])
    
    chain = prompt | llm_client
    
    try:
        start_time = time.time()
        response: AIMessage = chain.invoke({})
        processing_time = int((time.time() - start_time) * 1000)
        
        json_output_raw = response.content
        json_output = extract_json_from_llm_response(json_output_raw)
        
        parsed_data = json.loads(json_output)
        
        # 응답 검증
        if not isinstance(parsed_data, dict):
            raise ValueError("Invalid JSON structure")
        
        if "adjustments" not in parsed_data:
            raise ValueError("Missing 'adjustments' field")
        
        # 성공 결과 반환
        return {
            "success": True,
            "result": parsed_data,
            "processing_time_ms": processing_time,
            "fallback_used": False
        }
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 오류: {e}")
        return handle_json_parsing_error(cl_data, json_output_raw)
    
    except Exception as e:
        print(f"❌ LLM 호출 오류: {e}")
        return handle_llm_failure(cl_data)

def handle_json_parsing_error(cl_data: Dict, raw_response: str) -> Dict:
    """JSON 파싱 오류 처리"""
    print("🔧 JSON 구조 복구 시도 중...")
    
    try:
        # 간단한 복구 시도
        fixed_json = raw_response.strip()
        if fixed_json.startswith('```json'):
            fixed_json = fixed_json[7:]
        if fixed_json.endswith('```'):
            fixed_json = fixed_json[:-3]
        
        parsed_data = json.loads(fixed_json.strip())
        
        return {
            "success": True,
            "result": parsed_data,
            "processing_time_ms": 0,
            "fallback_used": True
        }
        
    except:
        # 복구 실패 시 기본 알고리즘
        return execute_fallback_algorithm(cl_data)

def handle_llm_failure(cl_data: Dict) -> Dict:
    """LLM 호출 실패 처리"""
    print("🔧 Fallback 알고리즘 실행 중...")
    return execute_fallback_algorithm(cl_data)

def execute_fallback_algorithm(cl_data: Dict) -> Dict:
    """기본 알고리즘 (비례 차감)"""
    
    members = cl_data['members']
    surplus = cl_data['surplus']
    
    if surplus <= 0:
        # 조정 불필요
        adjustments = [
            {
                "emp_no": member['emp_no'],
                "original_score": member['current_score'],
                "final_score": member['current_score'],
                "change_amount": 0,
                "change_type": "maintain",
                "reason": "조정 불필요"
            }
            for member in members
        ]
    else:
        # 단순 비례 차감
        adjustments = []
        reduction_per_person = surplus / len(members)
        
        for member in members:
            original = member['current_score']
            final = max(1.5, original - reduction_per_person)
            change = final - original
            
            adjustments.append({
                "emp_no": member['emp_no'],
                "original_score": original,
                "final_score": round(final, 2),
                "change_amount": round(change, 2),
                "change_type": "decrease" if change < 0 else "maintain",
                "reason": f"Fallback 비례 차감 (총 {surplus:.2f}점 분배)"
            })
    
    return {
        "success": True,
        "result": {
            "analysis_summary": f"Fallback 알고리즘으로 {surplus:.2f}점 조정",
            "adjustments": adjustments,
            "zero_sum_check": {
                "target_adjustment": surplus,
                "actual_adjustment": surplus,
                "achieved": True
            },
            "rejected_requests": []
        },
        "processing_time_ms": 0,
        "fallback_used": True
    }

# ================================================================
# 결과 저장 함수들
# ================================================================

def update_final_evaluation_reports(adjustments: List[Dict]) -> Dict:
    """final_evaluation_reports 테이블 배치 업데이트"""
    
    success_count = 0
    failed_updates = []
    
    with engine.connect() as connection:
        try:
            for adjustment in adjustments:
                emp_no = adjustment['emp_no']
                final_score = adjustment['final_score']
                original_score = adjustment['original_score']
                change_amount = adjustment['change_amount']
                reason = adjustment['reason']
                
                try:
                    # final_evaluation_reports 업데이트
                    query = text("""
                        UPDATE final_evaluation_reports 
                        SET 
                            score = :final_score,
                            adjustment_reason = :reason,
                            original_temp_score = :original_score,
                            adjustment_amount = :change_amount,
                            adjusted_by_module9 = 1
                        WHERE emp_no = :emp_no 
                        AND team_evaluation_id = (
                            SELECT fer.team_evaluation_id 
                            FROM final_evaluation_reports fer
                            JOIN team_evaluations te ON fer.team_evaluation_id = te.team_evaluation_id
                            WHERE fer.emp_no = :emp_no AND te.period_id = 4
                            LIMIT 1
                        )
                    """)
                    
                    result = connection.execute(query, {
                        "emp_no": emp_no,
                        "final_score": final_score,
                        "reason": reason,
                        "original_score": original_score,
                        "change_amount": change_amount
                    })
                    
                    if result.rowcount > 0:
                        success_count += 1
                        print(f"✅ {emp_no}: {original_score} → {final_score} 업데이트 완료")
                    else:
                        failed_updates.append(emp_no)
                        print(f"❌ {emp_no}: 업데이트 실패 (행 없음)")
                        
                except Exception as e:
                    failed_updates.append(emp_no)
                    print(f"❌ {emp_no}: 업데이트 실패 - {e}")
            
            connection.commit()
            
            return {
                "total_updates": len(adjustments),
                "successful_updates": success_count,
                "failed_updates": len(failed_updates),
                "updated_employees": [adj['emp_no'] for adj in adjustments if adj['emp_no'] not in failed_updates],
                "failed_employees": failed_updates
            }
            
        except Exception as e:
            connection.rollback()
            print(f"❌ 배치 업데이트 실패: {e}")
            return {
                "total_updates": len(adjustments),
                "successful_updates": 0,
                "failed_updates": len(adjustments),
                "updated_employees": [],
                "failed_employees": [adj['emp_no'] for adj in adjustments]
            }

# ================================================================
# 서브모듈 함수들
# ================================================================

def department_data_collection_submodule(state: Module9AgentState) -> Module9AgentState:
    """1. 부문 데이터 수집 서브모듈"""
    
    headquarter_id = state["headquarter_id"]
    period_id = state["period_id"]
    
    print(f"🔍 부문 데이터 수집 시작: 본부 {headquarter_id}")
    
    try:
        # 본부 내 모든 직원을 CL별로 조회
        cl_groups_raw = fetch_headquarter_cl_data(headquarter_id, period_id)
        
        # 각 CL별 처리 상황 분석
        department_data = {}
        
        for cl_name, members in cl_groups_raw.items():
            if members:
                surplus = calculate_cl_surplus(members)
                needs_adjustment = surplus > 0.05  # 0.05점 이상 차이나면 조정 필요
                
                # 상승 요청한 사람들 (팀장이 점수 올린 경우)
                members_with_requests = [
                    m['emp_no'] for m in members 
                    if m.get('captain_reason') and m.get('score_origin') == 'captain_adjusted'
                ]
                
                department_data[cl_name] = {
                    "surplus": surplus,
                    "needs_adjustment": needs_adjustment,
                    "member_count": len(members),
                    "members_with_requests": members_with_requests,
                    "members": members  # 상세 데이터 임시 보관
                }
                
                print(f"   {cl_name}: {len(members)}명, 초과분 {surplus:.2f}점, 조정 {'필요' if needs_adjustment else '불필요'}")
            else:
                department_data[cl_name] = {
                    "surplus": 0.0,
                    "needs_adjustment": False,
                    "member_count": 0,
                    "members_with_requests": [],
                    "members": []
                }
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"부문 데이터 수집 완료: 본부 {headquarter_id}")],
            "department_data": department_data
        })
        
        return updated_state
        
    except Exception as e:
        error_msg = f"부문 데이터 수집 실패: {str(e)}"
        print(f"❌ {error_msg}")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=error_msg)],
            "error_logs": state.get("error_logs", []) + [error_msg]
        })
        
        return updated_state

def cl_supervisor_execution_submodule(state: Module9AgentState) -> Module9AgentState:
    """2. CL별 Supervisor 실행 서브모듈"""
    
    headquarter_id = state["headquarter_id"]
    period_id = state["period_id"]
    department_data = state["department_data"]
    
    print(f"🤖 CL별 Supervisor 실행 시작")
    
    supervisor_results = {}
    total_processed = 0
    total_failed = 0
    
    try:
        for cl_name, cl_info in department_data.items():
            if cl_info.get("needs_adjustment", False):
                print(f"\n📊 {cl_name} 처리 중...")
                
                # Supervisor 입력 데이터 구성
                cl_data = {
                    "cl_group": cl_name,
                    "surplus": cl_info["surplus"],
                    "members": cl_info["members"]
                }
                
                # Supervisor LLM 호출
                result = call_supervisor_llm(cl_data, headquarter_id, period_id)
                
                if result["success"]:
                    # 결과 저장
                    adjustments = result["result"].get("adjustments", [])
                    
                    if adjustments:
                        # DB 업데이트 (실시간)
                        update_result = update_final_evaluation_reports(adjustments)
                        
                        supervisor_results[cl_name] = {
                            "success": True,
                            "adjustments_made": update_result["successful_updates"],
                            "zero_sum_achieved": result["result"].get("zero_sum_check", {}).get("achieved", False),
                            "processing_time_ms": result["processing_time_ms"],
                            "fallback_used": result["fallback_used"],
                            "update_details": update_result
                        }
                        
                        total_processed += update_result["successful_updates"]
                        total_failed += update_result["failed_updates"]
                    else:
                        # 조정 결과 없음 (모든 요청 거절)
                        supervisor_results[cl_name] = {
                            "success": True,
                            "adjustments_made": 0,
                            "zero_sum_achieved": True,
                            "processing_time_ms": result["processing_time_ms"],
                            "fallback_used": result["fallback_used"],
                            "note": "모든 조정 요청 거절됨"
                        }
                else:
                    supervisor_results[cl_name] = {
                        "success": False,
                        "adjustments_made": 0,
                        "zero_sum_achieved": False,
                        "processing_time_ms": 0,
                        "fallback_used": True,
                        "error": "Supervisor 실행 완전 실패"
                    }
                    total_failed += cl_info["member_count"]
                
            else:
                # 조정 불필요
                supervisor_results[cl_name] = {
                    "success": True,
                    "adjustments_made": 0,
                    "zero_sum_achieved": True,
                    "processing_time_ms": 0,
                    "fallback_used": False,
                    "note": "조정 불필요"
                }
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=f"CL별 Supervisor 실행 완료")],
            "supervisor_results": supervisor_results,
            "total_processed": total_processed,
            "total_failed": total_failed
        })
        
        return updated_state
        
    except Exception as e:
        error_msg = f"Supervisor 실행 실패: {str(e)}"
        print(f"❌ {error_msg}")
        
        updated_state = state.copy()
        updated_state.update({
            "messages": [HumanMessage(content=error_msg)],
            "supervisor_results": {},
            "total_processed": 0,
            "total_failed": sum(info.get("member_count", 0) for info in department_data.values()),
            "error_logs": state.get("error_logs", []) + [error_msg]
        })
        
        return updated_state

def batch_update_submodule(state: Module9AgentState) -> Module9AgentState:
    """3. 배치 업데이트 결과 정리 서브모듈"""
    
    supervisor_results = state["supervisor_results"]
    total_processed = state["total_processed"]
    total_failed = state["total_failed"]
    
    print(f"📊 배치 업데이트 결과 정리")
    
    # 전체 처리 결과 요약
    successful_cls = [cl for cl, result in supervisor_results.items() if result.get("success", False)]
    failed_cls = [cl for cl, result in supervisor_results.items() if not result.get("success", False)]
    
    update_results = {
        "successful_cls": successful_cls,
        "failed_cls": failed_cls,
        "total_adjustments": total_processed,
        "total_failures": total_failed,
        "summary": f"성공: {len(successful_cls)}개 CL, 실패: {len(failed_cls)}개 CL"
    }
    
    print(f"✅ 최종 결과: {update_results['summary']}")
    print(f"   조정 완료: {total_processed}명")
    print(f"   조정 실패: {total_failed}명")
    
    updated_state = state.copy()
    updated_state.update({
        "messages": [HumanMessage(content=f"모듈9 완료: {update_results['summary']}")],
        "update_results": update_results
    })
    
    return updated_state

# ================================================================
# LangGraph 워크플로우 구성
# ================================================================

# 모듈 9의 워크플로우 정의
module9_workflow = StateGraph(Module9AgentState)

# 노드 추가
module9_workflow.add_node("department_data_collection", department_data_collection_submodule)
module9_workflow.add_node("cl_supervisor_execution", cl_supervisor_execution_submodule)
module9_workflow.add_node("batch_update", batch_update_submodule)

# 엣지 정의 (순차 실행)
module9_workflow.add_edge(START, "department_data_collection")
module9_workflow.add_edge("department_data_collection", "cl_supervisor_execution")
module9_workflow.add_edge("cl_supervisor_execution", "batch_update")
module9_workflow.add_edge("batch_update", END)

# 그래프 컴파일
module9_graph = module9_workflow.compile()

# ================================================================
# 실행 함수들
# ================================================================

def run_module9_evaluation(headquarter_id: int, period_id: int = 4):
    """모듈9 부문 단위 CL별 정규화 실행"""
    
    print(f"🚀 모듈9 실행 시작: 본부 {headquarter_id} (period_id: {period_id})")
    
    # State 초기화
    state = Module9AgentState(
        messages=[HumanMessage(content=f"모듈9 시작: 본부 {headquarter_id}")],
        headquarter_id=headquarter_id,
        period_id=period_id,
        department_data={},
        supervisor_results={},
        update_results={},
        total_processed=0,
        total_failed=0,
        error_logs=[]
    )
    
    # 그래프 실행
    try:
        result = module9_graph.invoke(state)
        
        print("\n" + "="*60)
        print("✅ 모듈9 실행 완료!")
        print(f"📊 최종 결과:")
        print(f"   본부: {headquarter_id}")
        print(f"   처리 완료: {result['total_processed']}명")
        print(f"   처리 실패: {result['total_failed']}명")
        
        if result.get('update_results'):
            print(f"   결과: {result['update_results']['summary']}")
        
        if result.get('error_logs'):
            print(f"   오류 로그: {len(result['error_logs'])}건")
            for error in result['error_logs']:
                print(f"     - {error}")
        
        print("="*60)
        
        return result
        
    except Exception as e:
        print(f"\n❌ 모듈9 실행 실패: {e}")
        return None

def run_multiple_headquarters_module9(headquarter_ids: List[int], period_id: int = 4):
    """여러 본부 일괄 실행"""
    print(f"🚀 다중 본부 모듈9 실행: {len(headquarter_ids)}개 본부")
    
    results = {}
    total_processed = 0
    total_failed = 0
    
    for hq_id in headquarter_ids:
        print(f"\n{'='*50}")
        print(f"본부 {hq_id} 처리 중...")
        
        result = run_module9_evaluation(hq_id, period_id)
        results[hq_id] = result
        
        if result:
            total_processed += result.get('total_processed', 0)
            total_failed += result.get('total_failed', 0)
    
    print(f"\n🎯 전체 결과:")
    print(f"   처리된 본부: {len([r for r in results.values() if r is not None])}/{len(headquarter_ids)}")
    print(f"   총 조정 완료: {total_processed}명")
    print(f"   총 조정 실패: {total_failed}명")
    
    return results

def get_all_headquarters_with_data(period_id: int = 4) -> List[int]:
    """평가 데이터가 있는 모든 본부 ID 조회"""
    with engine.connect() as connection:
        query = text("""
            SELECT DISTINCT t.headquarter_id
            FROM teams t
            JOIN employees e ON t.team_id = e.team_id
            JOIN temp_evaluations te ON e.emp_no = te.TempEvaluation_empNo
            WHERE te.status = '완료'
            ORDER BY t.headquarter_id
        """)
        results = connection.execute(query).fetchall()
        return [row.headquarter_id for row in results]

def test_module9(headquarter_id: int = None, period_id: int = 4):
    """모듈9 테스트 실행"""
    if not headquarter_id:
        headquarters = get_all_headquarters_with_data(period_id)
        if headquarters:
            headquarter_id = headquarters[0]
            print(f"🧪 테스트 본부 자동 선택: {headquarter_id}")
        else:
            print("❌ 테스트할 본부가 없습니다")
            return
    
    return run_module9_evaluation(headquarter_id, period_id)

# ================================================================
# 실행 준비 완료
# ================================================================

print("✅ 모듈9 초기화 완료!")
print("\n🎯 사용 가능한 함수들:")
print("1. run_module9_evaluation(headquarter_id, period_id=4)     # 단일 본부 실행")
print("2. run_multiple_headquarters_module9([1,2,3], period_id=4) # 다중 본부 실행") 
print("3. test_module9()                                          # 테스트 실행")
print("4. get_all_headquarters_with_data()                        # 데이터 있는 본부 조회")

print("\n예시 실행:")
print("# test_module9()  # 테스트 실행")
print("# run_module9_evaluation(1, 4)  # 본부 1번 연말 평가")

# ================================================================
# 테스트 실행 (필요시 주석 해제)
# ================================================================

# 자동 테스트 실행 (주석 해제하여 사용)
# print("\n" + "="*60)
# print("🧪 자동 테스트 시작")
# print("="*60)
test_result = test_module9()

🚀 모듈9 초기화 중...
LLM Client initialized: gpt-4o-mini
✅ 모듈9 초기화 완료!

🎯 사용 가능한 함수들:
1. run_module9_evaluation(headquarter_id, period_id=4)     # 단일 본부 실행
2. run_multiple_headquarters_module9([1,2,3], period_id=4) # 다중 본부 실행
3. test_module9()                                          # 테스트 실행
4. get_all_headquarters_with_data()                        # 데이터 있는 본부 조회

예시 실행:
# test_module9()  # 테스트 실행
# run_module9_evaluation(1, 4)  # 본부 1번 연말 평가
❌ 테스트할 본부가 없습니다
